# Отток клиентов

В банке наблюдается отток клиентов каждый месяц. Это привело к тому, что маркетологи банка посчитали, что сохранять текущих клиентов является дешевле, чем привлекать новых.

Для уменьшения оттока клиентов необходимо спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Для решения этой задачи предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.

Для достижения наилучших результатов необходимо построить модель с предельно большим значением F1-меры, что обеспечит максимально точные прогнозы о возможном оттоке клиентов.

## Подготовка данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
try:
    data = pd.read_csv('/Users/arina200212yandex.ru/Desktop/Churn.csv')
except:
    data = pd.read_csv('/datasets/Churn.csv')

In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Для построения модели анализа оттока клиентов не нужны столбцы RowNumber, CustomerId и Surname. Эти столбцы не влияют на результаты модели и могут быть безопасно удалены из датасета.

In [ ]:
data = data.drop(["RowNumber","CustomerId","Surname"], axis = 1)

Проверим наличие дубликатов и пропусков.

In [ ]:
data.duplicated().sum()

0

In [ ]:
data.isna().sum()

CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

В столбце Tenure отмечены пропущенные значения, которые могут быть удалены из датасета, чтобы избежать потенциальных выбросов. Доля этих пропусков среди всех значений невелика, поэтому удаление их не окажет значительного влияния на дальнейший анализ данных.

In [ ]:
data = data.dropna(subset = ["Tenure"], axis = 0)

Подготовим данные с помощью ОНЕ.

In [ ]:
data = pd.get_dummies(data, drop_first=True)

In [ ]:
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Итак, после удаления ненужных столбцов и обработки пропущенных значений, данные готовы к дальнейшей работе и исследованию.

## Исследование задачи

Выберем целевой признак Exited, который отвечает за факт ухода клиента.

Для построения и обучения модели необходимо разделить выборку на три части: обучающую, валидационную и тестовую. 

In [ ]:
obuchenie, test = train_test_split(data, test_size=0.4, random_state=12345)
features = test.drop('Exited', axis=1)
target = test['Exited']

In [ ]:
features_valida, features_test, target_valida, target_test = train_test_split(features, target, test_size=0.5, random_state=12345)
features_obuchenie = obuchenie.drop('Exited', axis=1)
target_obuchenie = obuchenie['Exited']

Перед обучением модели логистической регрессии данные необходимо масштабировать.

In [ ]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']#численные признаки
scaler = StandardScaler()
scaler.fit(features_obuchenie[numeric])


StandardScaler()

После разделения данных на обучающую, валидационную и тестовую выборки, следует масштабировать признаки в каждой выборке.

In [ ]:
features_obuchenie[numeric] = scaler.transform(features_obuchenie[numeric])

In [ ]:
features_valida[numeric] = scaler.transform(features_valida[numeric])

In [ ]:
features_test[numeric] = scaler.transform(features_test[numeric])

Проверим баланс классов.

In [ ]:
target_obuchenie.value_counts(normalize = 1)

0    0.793546
1    0.206454
Name: Exited, dtype: float64

In [ ]:
target_valida.value_counts(normalize = 1)

0    0.792629
1    0.207371
Name: Exited, dtype: float64

Проведен анализ классов целевого признака показал, что наблюдается дисбаланс классов. Ответов 0 (клиент не ушел из банка) около 80%, а ответов 1 (клиент ушел из банка) – около 20%. 

Далее рассмотрены три модели для решения задачи классификации: "Дерево решений", "Случайный лес" и "Логистическая регрессия". 

Сначала рассмотрим модель "Дерево решений".

In [ ]:
f1_best= 0
best_depth = 0
for depth in range(1,25):
    model = DecisionTreeClassifier(max_depth=depth, random_state=1234)
    model.fit(features_obuchenie , target_obuchenie)
    predictions = model.predict(features_valida)
    f1 = f1_score(predictions, target_valida)
    probabilities_one_valid = model.predict_proba(features_valida)[:, 1]
    AUC = roc_auc_score(target_valida, probabilities_one_valid)
    if f1 > f1_best:
        f1_best = f1
        best_depth = depth
print("Глубина дерева:", best_depth, "F1:", f1_best,'AUC-ROC',AUC)

Глубина дерева: 7 F1: 0.580952380952381 AUC-ROC 0.6797786314764468


Рассмотрим модель "Случаный лес"

In [ ]:
f1_best= 0
best_depth = 0
for depth in range(1,20):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=1234)
            model.fit(features_obuchenie , target_obuchenie)
            predictions = model.predict(features_valida)
            f1 = f1_score(predictions, target_valida)
            probabilities_one_valid = model.predict_proba(features_valida)[:, 1]
            AUC = roc_auc_score(target_valida, probabilities_one_valid)
            if f1 > f1_best:
                f1_best = f1
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "Количество деревьев:", best_est,"F1:", f1_best,'AUC-ROC',AUC,sample)

Глубина дерева: 16 Количество деревьев: 15 F1: 0.6026936026936026 AUC-ROC 0.8650933167911321 4


Рассмотрим модель "Логическая регрессия"

In [ ]:
model = LogisticRegression()
model.fit(features_obuchenie , target_obuchenie)
predictions = model.predict(features_valida)
f1 = f1_score(predictions, target_valida)
probabilities_one_valid = model.predict_proba(features_valida)[:, 1]
AUC = roc_auc_score(target_valida, probabilities_one_valid)
print("F1:", f1,'AUC-ROC',AUC)

F1: 0.30400000000000005 AUC-ROC 0.773663293800172


Изучение моделей "Дерево решений", "Случайный лес" и "Логистическая регрессия" показало, что наилучшая модель по метрике точности – "Случайный лес". Однако F1-мера для всех трех моделей достаточно низкая, что говорит о низком качестве моделей в целом.

## Борьба с дисбалансом

1 способ борьбы - upsampling, то есть увеличение выборки.

In [ ]:

def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

In [ ]:
features_upsampled_train, target_upsampled_train = upsample(features_obuchenie, target_obuchenie, repeat=4)

In [ ]:
target_upsampled_train.value_counts(normalize = 1)

1    0.509964
0    0.490036
Name: Exited, dtype: float64

2 способ борьбы - downsampling, то есть уменьшение выборки.

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

In [ ]:
features_downsampled_train, target_downsampled_train = downsample(features_obuchenie, target_obuchenie, fraction=0.25)

In [ ]:
target_downsampled_train.value_counts(normalize = 1)

1    0.509964
0    0.490036
Name: Exited, dtype: float64

Для проверки моделей на данных, которые были сбалансированы методом upsample, сначала рассмотрим модель "Дерево решений".

In [ ]:
f1_best= 0
best_depth = 0
for depth in range(1,25):
    model = DecisionTreeClassifier(max_depth=depth, random_state=1234)
    model.fit(features_upsampled_train, target_upsampled_train)
    predictions = model.predict(features_valida)
    f1 = f1_score(predictions, target_valida)
    probabilities_one_valid = model.predict_proba(features_valida)[:, 1]
    AUC = roc_auc_score(target_valida, probabilities_one_valid)
    if f1 > f1_best:
        f1_best = f1
        best_depth = depth
print("Глубина дерева:", best_depth, "F1:", f1_best,'AUC-ROC',AUC)

Глубина дерева: 5 F1: 0.5735449735449736 AUC-ROC 0.6654750882179153


Случаный лес.

In [ ]:
f1_best= 0
best_depth = 0
for depth in range(1,20):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=1234)
            model.fit(features_upsampled_train, target_upsampled_train)
            predictions = model.predict(features_valida)
            f1 = f1_score(predictions, target_valida)
            probabilities_one_valid = model.predict_proba(features_valida)[:, 1]
            AUC = roc_auc_score(target_valida, probabilities_one_valid)
            if f1 > f1_best:
                f1_best = f1
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "Количество деревьев:", best_est,"F1:", f1_best,'AUC-ROC',AUC,sample)

Глубина дерева: 18 Количество деревьев: 45 F1: 0.6444159178433889 AUC-ROC 0.8680808530769047 4


Линейная регрессия.

In [ ]:
model = LogisticRegression()
model.fit(features_upsampled_train, target_upsampled_train)
predictions = model.predict(features_valida)
f1 = f1_score(predictions, target_valida)
probabilities_one_valid = model.predict_proba(features_valida)[:, 1]
AUC = roc_auc_score(target_valida, probabilities_one_valid)
print("F1:", f1,'AUC-ROC',AUC)

F1: 0.5085972850678734 AUC-ROC 0.777854680197402


Для проверки моделей на данных, которые были сбалансированы методом downsample, рассмотрим модель "Дерево решений".

In [ ]:
f1_best= 0
best_depth = 0
for depth in range(1,25):
    model = DecisionTreeClassifier(max_depth=depth, random_state=1234)
    model.fit(features_downsampled_train, target_downsampled_train)
    predictions = model.predict(features_valida)
    f1 = f1_score(predictions, target_valida)
    probabilities_one_valid = model.predict_proba(features_valida)[:, 1]
    AUC = roc_auc_score(target_valida, probabilities_one_valid)
    if f1 > f1_best:
        f1_best = f1
        best_depth = depth
print("Глубина дерева:", best_depth, "F1:", f1_best,'AUC-ROC',AUC)

Глубина дерева: 6 F1: 0.5636704119850188 AUC-ROC 0.7048247882677995


Случаный лес.

In [ ]:
f1_best= 0
best_depth = 0
for depth in range(1,20):
    for est in range(5,50,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=1234)
            model.fit(features_downsampled_train, target_downsampled_train)
            predictions = model.predict(features_valida)
            f1 = f1_score(predictions, target_valida)
            probabilities_one_valid = model.predict_proba(features_valida)[:, 1]
            AUC = roc_auc_score(target_valida, probabilities_one_valid)
            if f1 > f1_best:
                f1_best = f1
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "Количество деревьев:", best_est,"F1:", f1_best,'AUC-ROC',AUC,sample)

Глубина дерева: 16 Количество деревьев: 20 F1: 0.6035502958579881 AUC-ROC 0.8628917805016778 4


Линейная регрессия.

In [ ]:
model = LogisticRegression()
model.fit(features_downsampled_train, target_downsampled_train)
predictions = model.predict(features_valida)
f1 = f1_score(predictions, target_valida)
probabilities_one_valid = model.predict_proba(features_valida)[:, 1]
AUC = roc_auc_score(target_valida, probabilities_one_valid)
print("F1:", f1,'AUC-ROC',AUC)

F1: 0.5044883303411131 AUC-ROC 0.7770245022153419


Благодаря сбалансированию классов мы смогли улучшить F1-меры для всех моделей и достигнуть значения выше 0.59. 

Для проведения тестирования выберем модель "Случайный лес", так как она показала лучший результат по метрикам качества. Оптимальными гиперпараметрами для этой модели оказались: max_depth = 18, n_estimators = 45, min_samples_leaf = 4, random_state = 1234. 

Именно эта модель будет использоваться для прогнозирования ухода клиентов из банка на новых данных.

## Тестирование модели

In [ ]:
model = RandomForestClassifier(max_depth=18, n_estimators=45, min_samples_leaf=4, random_state=1234)
model.fit(features_upsampled_train, target_upsampled_train)
predictions = model.predict(features_test)
f1 = f1_score(predictions, target_test)
probabilities_one_valid = model.predict_proba(features_test)[:, 1]
AUC = roc_auc_score(target_test, probabilities_one_valid)

print("F1:",f1,'AUC-ROC',AUC)

F1: 0.5984251968503936 AUC-ROC 0.855653756879915


После проверки модели "Дерево решений" на данных, которые были сбалансированы методом upsampled, была получена F1-мера 0.598, что выше, чем без учета балансировки классов.

В целом, сравнив три модели ("Дерево решений", "Случайный лес" и "Логистическая регрессия"), можно заключить, что наилучший результат показала модель "Случайный лес" с гиперпараметрами: max_depth = 18, n_estimators = 45, min_samples_leaf = 4, random_state = 1234. Эти параметры позволили достичь F1-меры на тестовой выборке чуть выше, чем 0.59.

Тем не менее, для повышения точности модели можно продолжать экспериментировать с различными значениями гиперпараметров, а также использовать другие алгоритмы обучения.